yaml作成

In [3]:
import yaml
from pathlib import Path

# 製品カテゴリのリスト
product_names = [
    "chips", "chocolate", "cookie", "fried_dough_cookie", "marshmallow", "ramune",
    "rice_cracker",
    "sandwich_cookie", "small_ramune"
    , "snack", "soft_candy", "stick_snack"
]

# 出力リストを作成する関数
def generate_directories(product_name):
    source_dir = Path(f"/data01/snacks/{product_name}")
    test_dir = Path(f"/data01/snacks/test/{product_name}")

    normal_dirs = []
    abnormal_dirs = {}
    test_dirs = []  # normal_test_dir用のリスト

    # source_dirのすべてのサブフォルダ（製品名フォルダ）を探索
    for product_dir in source_dir.glob("*/"):  # 製品ごとのフォルダを探索
        current_product_name = product_dir.name

        # "__templete" フォルダを無視する
        if current_product_name == "__templete":
            continue

        # 正常なディレクトリを追加
        normal_dir = f"{current_product_name}/ok"
        normal_dirs.append(normal_dir)

        # 欠陥フォルダのパスを追加: "ok"以外のフォルダを欠陥として扱う
        for defect_dir in product_dir.glob("*"):
            if defect_dir.is_dir() and defect_dir.name != "ok":
                defect_type = defect_dir.name
                if defect_type not in abnormal_dirs:
                    abnormal_dirs[defect_type] = []
                abnormal_dirs[defect_type].append(f"{current_product_name}/{defect_type}")

    # test_dirのすべてのサブフォルダ（製品名フォルダ）を探索
    for product_dir in test_dir.glob("*/"):  # テスト用の製品フォルダを探索
        current_product_name = product_dir.name

        # テスト用の正常ディレクトリを追加
        test_ok_dir = test_dir / current_product_name / "ok"  # 正しいtest_dirを参照
        test_dirs.append(str(test_ok_dir))

    return normal_dirs, abnormal_dirs, test_dirs

# 結果をYAML形式に出力する関数
def generate_yaml_file(normal_dirs, abnormal_dirs, test_dirs, product_name):
    yaml_output = 'class_path: anomalib.data.Folder\n'
    yaml_output += 'init_args:\n'
    yaml_output += f'  name: "SnackDataset"\n'
    yaml_output += f'  root: "/data01/snacks/{product_name}"\n'

    # normal_dirの出力
    yaml_output += '  normal_dir:\n'
    for normal in normal_dirs:
        yaml_output += f'    - "{normal}"\n'

    # abnormal_dirの出力
    yaml_output += '  abnormal_dir:\n'
    for dirs in abnormal_dirs.values():
        for defect_dir in dirs:
            yaml_output += f'    - "{defect_dir}"\n'

    # normal_test_dirの出力（testディレクトリの製品名ごとのokフォルダ）
    yaml_output += '  normal_test_dir:\n'
    for test in test_dirs:
        yaml_output += f'    - "{test}"\n'

    # その他の設定
    yaml_output += '  extensions: [".png"]\n'
    yaml_output += '  image_size: [256, 256]\n'
    yaml_output += '  train_batch_size: 32\n'
    yaml_output += '  eval_batch_size: 32\n'
    yaml_output += '  num_workers: 8\n'
    yaml_output += '  task: classification\n'
    yaml_output += '  test_split_mode: from_dir\n'
    yaml_output += '  test_split_ratio: 0.2\n'
    yaml_output += '  val_split_mode: same_as_test\n'
    yaml_output += '  val_split_ratio: 0.5\n'
    yaml_output += '  seed: null\n'

    # ファイルに書き込み
    with open(f"yaml/{product_name}_classification.yaml", "w") as file:
        file.write(yaml_output)
        # YAMLファイルが存在しない場合にのみ作成
        yaml_file_path = Path(f"yaml/{product_name}_classification.yaml")
        if not yaml_file_path.exists():
            generate_yaml_file(normal_dirs, abnormal_dirs, test_dirs, product_name)
# 各製品カテゴリに対してYAMLファイルを生成
for product_name in product_names:
    normal_dirs, abnormal_dirs, test_dirs = generate_directories(product_name)
    generate_yaml_file(normal_dirs, abnormal_dirs, test_dirs, product_name)

# 結果の確認
print("すべてのYAMLファイルが生成されました。")


すべてのYAMLファイルが生成されました。


合計枚数カウント

カテゴリチェック